# Sentiment Analysis LSTM and GRU

In [1]:
import tensorflow as tf

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
import numpy as np
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Embedding, Reshape, Activation, Input, Lambda, Dense, GRU, LSTM, CuDNNLSTM, CuDNNGRU, Dropout, Conv1D, MaxPooling1D, Flatten, concatenate
from tensorflow.python.keras.layers.merge import Dot
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.utils.data_utils import get_file
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import skipgrams, pad_sequences
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.python.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
# from tensorflow.python.keras.optimizers import Adam
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import gensim

In [9]:
from tensorflow.python.keras.datasets import imdb

In [10]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer # takes into consideration the morphological analysis of the words
from nltk.stem.porter import PorterStemmer # cutting off the end or the beginning of the word.

In [11]:
remove_terms = punctuation + '0123456789'

In [12]:
num_words = 20000

Dividing Datasets

In [13]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/vormenesse/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/vormenesse/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is 

In [14]:
print(len(X_train),'train_data')
print(len(X_test),'test_data')

25000 train_data
25000 test_data


In [15]:
print('text already in tokens:')
len(X_train[0]),len(X_train[1]),len(X_train[2])

text already in tokens:


(218, 189, 141)

Setting some parameters

In [16]:
max_len = 256
embedding_size = 10
batch_size = 128
n_epochs = 10

In [17]:
pad = 'pre' #'post' # with you want to pad pre or post the text

In [18]:
X_train_pad = pad_sequences(X_train, maxlen=max_len, padding=pad, truncating=pad)
X_test_pad = pad_sequences(X_test, maxlen=max_len, padding=pad, truncating=pad)

- Creating Model - 

In [14]:
model = Sequential()
model.add(Embedding(
    input_dim=num_words,output_dim=embedding_size,input_length=max_len,
    name='layer_embedding'                
    ))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(250,activation='relu'))
model.add(Dense(1,activation='sigmoid',name='classification'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 256, 10)           200000    
_________________________________________________________________
conv1d (Conv1D)              (None, 256, 128)          3968      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 128, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 128, 128)          49280     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 64, 128)           49280     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 32, 128)           0

In [16]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
callback_early_stopping = EarlyStopping(monitor='val_loss',patience=5,verbose=1)

In [18]:
model.fit(
    X_train_pad, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_split=0.05,
    callbacks=[callback_early_stopping]
)

Epoch 1/10
186/186 [==============================] - 8s 45ms/step - loss: 0.5546 - accuracy: 0.6793 - val_loss: 0.3523 - val_accuracy: 0.8528
Epoch 2/10
186/186 [==============================] - 5s 25ms/step - loss: 0.2961 - accuracy: 0.8780 - val_loss: 0.2993 - val_accuracy: 0.8784
Epoch 3/10
186/186 [==============================] - 4s 24ms/step - loss: 0.2267 - accuracy: 0.9096 - val_loss: 0.4257 - val_accuracy: 0.8376
Epoch 4/10
186/186 [==============================] - 4s 24ms/step - loss: 0.1879 - accuracy: 0.9268 - val_loss: 0.2868 - val_accuracy: 0.9016
Epoch 5/10
186/186 [==============================] - 4s 24ms/step - loss: 0.1583 - accuracy: 0.9384 - val_loss: 0.3319 - val_accuracy: 0.8824
Epoch 6/10
186/186 [==============================] - 4s 24ms/step - loss: 0.1336 - accuracy: 0.9504 - val_loss: 0.3374 - val_accuracy: 0.8808
Epoch 7/10
186/186 [==============================] - 4s 24ms/step - loss: 0.1143 - accuracy: 0.9579 - val_loss: 0.3179 - val_accuracy: 0.8880

In [20]:
eval_ = model.evaluate(X_test_pad, y_test)

782/782 [==============================] - 4s 5ms/step - loss: 0.4162 - accuracy: 0.8702


In [21]:
print('Loss', eval_[0])
print('Accuracy', eval_[1])

Loss 0.416238933801651
Accuracy 0.8701599836349487


### Combining all approaches

In [48]:
model = Sequential()
model.add(Embedding(
    input_dim=num_words,output_dim=embedding_size,input_length=max_len,
    name='layer_embedding'                
    ))
model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.5))
model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.5))
model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.5))
model.add(CuDNNLSTM(256))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid',name='classification'))

In [49]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 256, 10)           200000    
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 256, 128)          6528      
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 85, 128)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 85, 128)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 85, 256)           164096    
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 28, 256)           0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 28, 256)          

In [50]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [51]:
callback_early_stopping = EarlyStopping(monitor='val_loss',patience=5,verbose=1)

In [52]:
model.fit(
    X_train_pad, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_split=0.05,
    callbacks=[callback_early_stopping]
)

Epoch 1/10
186/186 [==============================] - 12s 64ms/step - loss: 0.5515 - accuracy: 0.6832 - val_loss: 0.3557 - val_accuracy: 0.8488
Epoch 2/10
186/186 [==============================] - 6s 32ms/step - loss: 0.2995 - accuracy: 0.8776 - val_loss: 0.2986 - val_accuracy: 0.8832
Epoch 3/10
186/186 [==============================] - 6s 31ms/step - loss: 0.2226 - accuracy: 0.9135 - val_loss: 0.3527 - val_accuracy: 0.8536
Epoch 4/10
186/186 [==============================] - 6s 30ms/step - loss: 0.1741 - accuracy: 0.9349 - val_loss: 0.3993 - val_accuracy: 0.8376
Epoch 5/10
186/186 [==============================] - 6s 30ms/step - loss: 0.1434 - accuracy: 0.9477 - val_loss: 0.2719 - val_accuracy: 0.8992
Epoch 6/10
186/186 [==============================] - 6s 30ms/step - loss: 0.1163 - accuracy: 0.9577 - val_loss: 0.3394 - val_accuracy: 0.8744
Epoch 7/10
186/186 [==============================] - 6s 30ms/step - loss: 0.0941 - accuracy: 0.9672 - val_loss: 0.3127 - val_accuracy: 0.893

In [53]:
eval_ = model.evaluate(X_test_pad, y_test)

782/782 [==============================] - 7s 8ms/step - loss: 0.6871 - accuracy: 0.8304


In [54]:
print('Loss', eval_[0])
print('Accuracy', eval_[1])

Loss 0.6870551705360413
Accuracy 0.8304399847984314


### Trying Something New

In [66]:
conv = []
filter_sizes = [3,4,5]

embedding_layer = Embedding(
    input_dim=num_words,output_dim=embedding_size,input_length=max_len,
    name='layer_embedding'                
    )

In [67]:
seq_input = Input(shape=(max_len,),dtype='int32')
embed_seq = embedding_layer(seq_input)

In [68]:
for f in filter_sizes:
    _conv = Conv1D(filters=128, kernel_size=f, activation='relu')(embed_seq)
    _pool = MaxPooling1D(5)(_conv)
    conv.append(_pool)

In [69]:
_concat = concatenate(conv, axis=1) # axis = 1 for performing the operation in columns with same size
_conv1 = Conv1D(filters=128, kernel_size=5, activation='relu')(_concat)
_pool1 = MaxPooling1D(5)(_conv1)
_drop1 = Dropout(0.5)(_pool1)
_conv2 = Conv1D(filters=128, kernel_size=5, activation='relu')(_drop1)
_pool2 = MaxPooling1D(5)(_conv2)
_drop2 = Dropout(0.5)(_pool2)
_flat = Flatten()(_drop2)
_drop3 = Dropout(0.5)(_flat)
_dense = Dense(128,activation='relu')(_drop3)
preds = Dense(1, activation='sigmoid')(_dense)
model = Model(seq_input,preds)

In [70]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
layer_embedding (Embedding)     (None, 256, 10)      200000      input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 254, 128)     3968        layer_embedding[0][0]            
__________________________________________________________________________________________________
conv1d_23 (Conv1D)              (None, 253, 128)     5248        layer_embedding[0][0]            
______________________________________________________________________________________________

In [71]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
callback_early_stopping = EarlyStopping(monitor='val_loss',patience=5,verbose=1)

In [73]:
model.fit(
    X_train_pad, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_split=0.05,
    callbacks=[callback_early_stopping]
)

Epoch 1/10
186/186 [==============================] - 15s 82ms/step - loss: 0.6402 - accuracy: 0.5877 - val_loss: 0.4459 - val_accuracy: 0.8112
Epoch 2/10
186/186 [==============================] - 7s 38ms/step - loss: 0.3744 - accuracy: 0.8373 - val_loss: 0.3283 - val_accuracy: 0.8680
Epoch 3/10
186/186 [==============================] - 7s 36ms/step - loss: 0.2615 - accuracy: 0.8992 - val_loss: 0.3313 - val_accuracy: 0.8544
Epoch 4/10
186/186 [==============================] - 7s 38ms/step - loss: 0.2009 - accuracy: 0.9243 - val_loss: 0.3179 - val_accuracy: 0.8784
Epoch 5/10
186/186 [==============================] - 7s 37ms/step - loss: 0.1634 - accuracy: 0.9392 - val_loss: 0.3001 - val_accuracy: 0.8864
Epoch 6/10
186/186 [==============================] - 7s 38ms/step - loss: 0.1300 - accuracy: 0.9535 - val_loss: 0.3565 - val_accuracy: 0.8776
Epoch 7/10
186/186 [==============================] - 7s 39ms/step - loss: 0.1083 - accuracy: 0.9620 - val_loss: 0.4747 - val_accuracy: 0.860

In [74]:
eval_ = model.evaluate(X_test_pad, y_test)

782/782 [==============================] - 7s 9ms/step - loss: 0.6264 - accuracy: 0.8431


In [76]:
print('Loss', eval_[0])
print('Accuracy', eval_[1])

Loss 0.626409113407135
Accuracy 0.8430799841880798


### Different

In [77]:
conv = []
filter_sizes = [3,4,5]

embedding_layer = Embedding(
    input_dim=num_words,output_dim=embedding_size,input_length=max_len,
    name='layer_embedding'                
    )

In [78]:
seq_input = Input(shape=(max_len,),dtype='int32')
embed_seq = embedding_layer(seq_input)

In [79]:
for f in filter_sizes:
    _conv = Conv1D(filters=256, kernel_size=f, activation='relu')(embed_seq)
    _pool = MaxPooling1D(5)(_conv)
    conv.append(_pool)

In [80]:
_concat = concatenate(conv, axis=1) # axis = 1 for performing the operation in columns with same size
_conv1 = Conv1D(filters=128, kernel_size=5, activation='relu')(_concat)
_pool1 = MaxPooling1D(5)(_conv1)
_drop1 = Dropout(0.5)(_pool1)
_conv2 = Conv1D(filters=128, kernel_size=5, activation='relu')(_drop1)
_pool2 = MaxPooling1D(5)(_conv2)
_drop2 = Dropout(0.5)(_pool2)
_flat = Flatten()(_drop2)
_drop3 = Dropout(0.5)(_flat)
_dense = Dense(128,activation='relu')(_drop3)
preds = Dense(1, activation='sigmoid')(_dense)
model = Model(seq_input,preds)

In [81]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
layer_embedding (Embedding)     (None, 256, 10)      200000      input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_27 (Conv1D)              (None, 254, 256)     7936        layer_embedding[0][0]            
__________________________________________________________________________________________________
conv1d_28 (Conv1D)              (None, 253, 256)     10496       layer_embedding[0][0]            
____________________________________________________________________________________________

In [82]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [83]:
callback_early_stopping = EarlyStopping(monitor='val_loss',patience=5,verbose=1)

In [84]:
model.fit(
    X_train_pad, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_split=0.05,
    callbacks=[callback_early_stopping]
)

Epoch 1/10
186/186 [==============================] - 17s 91ms/step - loss: 0.6478 - accuracy: 0.5723 - val_loss: 0.4153 - val_accuracy: 0.8424
Epoch 2/10
186/186 [==============================] - 11s 61ms/step - loss: 0.3291 - accuracy: 0.8612 - val_loss: 0.2887 - val_accuracy: 0.8896
Epoch 3/10
186/186 [==============================] - 11s 57ms/step - loss: 0.2366 - accuracy: 0.9088 - val_loss: 0.2767 - val_accuracy: 0.8840
Epoch 4/10
186/186 [==============================] - 11s 59ms/step - loss: 0.1843 - accuracy: 0.9303 - val_loss: 0.3708 - val_accuracy: 0.8504
Epoch 5/10
186/186 [==============================] - 11s 60ms/step - loss: 0.1517 - accuracy: 0.9434 - val_loss: 0.3528 - val_accuracy: 0.8584
Epoch 6/10
186/186 [==============================] - 11s 60ms/step - loss: 0.1204 - accuracy: 0.9567 - val_loss: 0.3695 - val_accuracy: 0.8776
Epoch 7/10
186/186 [==============================] - 11s 57ms/step - loss: 0.0935 - accuracy: 0.9680 - val_loss: 0.4366 - val_accuracy:

In [85]:
eval_ = model.evaluate(X_test_pad, y_test)

782/782 [==============================] - 9s 11ms/step - loss: 0.4778 - accuracy: 0.8456


In [86]:
print('Loss', eval_[0])
print('Accuracy', eval_[1])

Loss 0.477815181016922
Accuracy 0.8456000089645386


# Auto Keras

In [19]:
%load_ext autoreload
%autoreload 2

In [20]:
import autokeras as ak

In [52]:
clf = ak.StructuredDataClassifier(max_trials=2000)
clf.fit(X_train_pad, y_train, epochs=20)

Trial 2 Complete [00h 00m 47s]
val_accuracy: 0.5026167631149292

Best val_accuracy So Far: 0.5346215963363647
Total elapsed time: 00h 01m 18s
INFO:tensorflow:Oracle triggered exit
Epoch 1/20
782/782 [==============================] - 4s 3ms/step - loss: 0.7620 - accuracy: 0.5062
Epoch 2/20
782/782 [==============================] - 2s 3ms/step - loss: 0.6927 - accuracy: 0.5266
Epoch 3/20
782/782 [==============================] - 2s 3ms/step - loss: 0.6878 - accuracy: 0.5383
Epoch 4/20
782/782 [==============================] - 2s 3ms/step - loss: 0.6848 - accuracy: 0.5449
Epoch 5/20
782/782 [==============================] - 2s 3ms/step - loss: 0.6828 - accuracy: 0.5542
Epoch 6/20
782/782 [==============================] - 2s 3ms/step - loss: 0.6799 - accuracy: 0.5575
Epoch 7/20
782/782 [==============================] - 2s 3ms/step - loss: 0.6798 - accuracy: 0.5600
Epoch 8/20
782/782 [==============================] - 2s 3ms/step - loss: 0.6762 - accuracy: 0.5646
Epoch 9/20
782/782 [

In [53]:
results = clf.predict(X_test_pad)

782/782 [==============================] - 2s 2ms/step


In [54]:
from sklearn.metrics import accuracy_score

In [55]:
accuracy_score(y_test,results)

0.51856